In [1]:
import tensorflow as tf
import sonnet as snt
import dnc
import trfl
from dnc import dnc
# tf.enable_eager_execution()

/home/act65/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/act65/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [2]:
env = []
# we could make it partial info by removing
# the image of the player from the observations

In [9]:
def local_idx(indexes, values):
    """
    
    """
    # BUG indexes is not differentiable!?
    
    
    B = tf.shape(indexes)[0]
    H = tf.shape(values)[0]
    W = tf.shape(values)[1]
    D = tf.shape(values)[2]
    
    # scale by shape of values
    x_idx = indexes[:, 1]*tf.cast(W, tf.float32)
    y_idx = indexes[:, 0]*tf.cast(H, tf.float32)

    # cast to ints and construct indexes
    idx = tf.stack([x_idx, y_idx], axis=-1)
    idx = tf.cast(tf.round(idx), tf.int32)
    # idx = [[x_idx, y_idx], ...]

    values = tf.image.resize_image_with_crop_or_pad(
        values, target_height=H+2, target_width=W+2)
    
    # TODO want a more general way to do this.
    # for kernels of size N
    x_shift = tf.stack([ #tf.zeros([B], dtype=tf.int32), 
                    tf.ones([B], dtype=tf.int32), 
                    tf.zeros([B], dtype=tf.int32)], 
                       axis=-1)
    
    y_shift = tf.stack([#tf.zeros([B], dtype=tf.int32), 
                        tf.zeros([B], dtype=tf.int32), 
                        tf.ones([B], dtype=tf.int32)], 
                       axis=-1)
    
    c = tf.gather_nd(values, idx)
    l = tf.gather_nd(values, idx - x_shift)
    r = tf.gather_nd(values, idx + x_shift)
    t = tf.gather_nd(values, idx - y_shift)
    b = tf.gather_nd(values, idx + y_shift)
    
    # TODO use a guassian kernel instead.
    # and expore the variance as an argument.
    v = 0.5*c + 0.1*l + 0.1*r + 0.1*t + 0.1*b
    
    return v
    
# B, H, W, D = (50, 12, 12, 8)
# #     kernel = gaussian_kernel(1, indexes, 1.0)
# indexes = tf.nn.sigmoid(tf.random_normal([B, 3]))  # x and y (and ?)
# values = tf.random_normal([H, W, D])
# local_idx(indexes, values)

class SimpleMemory2D(snt.AbstractModule):
    """
    A simple memory. Does not learn how to write to memory.
    Only to read.
    The content of the memory is trained via GD on the task.
    
    Want to test. 
    Q? If we give an agent 2D memory, and constrain it to be locally indexed,
    then is the index an estimate of the agents location?
    (and thus does structure of the memory match the environment?) 
    """
    def __init__(self, height, width, dims, name='Memory2d'):
        super(SimpleMemory2D, self).__init__(name=name)
        
        self._height = height
        self._width = width
        self._dims = dims
        
        with self._enter_variable_scope():
            self.memory = tf.get_variable(name='memory', shape=[height, width, dims])
        
    def _build(self, inputs):
        return local_idx(indexes=inputs, values=self.memory)

In [10]:
mem = Memory2D(8,8,1)
mem(tf.random_normal([50, 3]))

<tf.Tensor 'Memory2d_3_1/add_7:0' shape=(50, 1) dtype=float32>

In [ ]:
class DNC(snt.RNNCore):
    def __init__(self, name='dnc'):
        super(DNC, self).__init__(name=name)

        with self._enter_variable_scope():
            self._controller = snt.LSTM(**controller_config)
            self._memory = 

    def _build(self, x):

In [125]:
def gaussian_kernel(size: int,
                    mean: float,
                    std: float,
                   ):
    """Makes 2D gaussian Kernel for convolution."""

    d = tf.distributions.Normal(mean, std)
    vals = d.prob(tf.range(start=-size, limit=size+1, dtype=tf.float32))
    gauss_kernel = tf.einsum('i,j->ij', vals, vals)
    return gauss_kernel / tf.reduce_sum(gauss_kernel)

In [34]:
T = 12
B = 1
dims = 1

observations = tf.placeholder(shape=[T, B, dims], dtype=tf.float32)
actions = tf.placeholder(shape=[T, B], dtype=tf.float32)
rewards = tf.placeholder(shape=[T, B], dtype=tf.float32)

In [ ]:
q_values, _, pos = tf.nn.dynamic_rnn(dnc_core, input_seq, init_state, time_major=True)

losses = [qlearning(q_tm1, a_tm1, r_t, pcont_t, q_t) 
          for q_tm1, a_tm1, r_t, pcont_t, q_t in zip(q_values[:-1], actions, rewards, ???, q_values[1:])]

In [ ]:
class MemoryAccess2D(snt.RNNCore):
    def __init__(self, height, width, name='Memory2d'):
        super(MemoryAccess2D, self).__init__(name=name)
        
        self._height = height
        self._width = width
        self._dims = dims
        
    def _build(self, inputs, state):
        """
        Use inputs to look up a value from state.
        
        Args:
            inputs ():
            state (tf.tensor): 2d tensors. [width x height x dims]
            
        Returns:
            (tf.tensor): the value looked up
        """
        inputs = self.parse_inputs(inputs)
        
        state = self.write_local(index=inputs, value=state)
        output = self.read_local(index=inputs, value=state)
        
        return output, state
        
    @property
    def state_size(self):
        """Returns a tuple of the shape of the state tensors."""
        return AccessState2d(
            memory=tf.TensorShape([self._width, self._height, self._dims]))

    @property
    def output_size(self):
        """Returns the output shape."""
        return tf.TensorShape([self._dims])

In [56]:
x = tf.Variable(tf.random_normal([1, 8]))

<tf.Tensor 'ArgMax_1:0' shape=(1,) dtype=int64>